In [9]:
import os
import scipy.io
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pickle
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt


In [12]:
def load_cache(path, encoding="latin-1", fix_imports=True):
        """
        encoding latin-1 is default for Python2 compatibility
        """
        with open(path, "rb") as f:
            return pickle.load(f, encoding=encoding, fix_imports=True)
        
def create_unique_ann_list():
        
        for data in dataset['samples']:
            new_ann = data['annotation']
            if new_ann not in cars_annotations:
                cars_annotations.append(new_ann)

def convert_ann_to_num(ann):
    if ann not in cars_annotations:
        cars_annotations.append(ann)
    return cars_annotations.index(ann)

In [18]:
cars_annotations = []
split = load_cache('../data/BoxCars/classification_splits.pkl')['hard']
dataset = load_cache('../data/BoxCars/dataset.pkl')
atlas = load_cache('../data/BoxCars/atlas.pkl')

create_unique_ann_list()
print('Number of annotations ' + str(len(cars_annotations)))

Number of annotations 693


In [38]:
def get_unique_list(li):
    return list(set(li))

def create_unique_multitask_list():    
    make = []
    mode = []
    submodel = []
    generation = []
    
    for new_ann in cars_annotations:
        t_make,t_model,t_submodel,t_generation = new_ann.split()
        make.append(t_make)
        mode.append(t_model)
        submodel.append(t_submodel)
        generation.append(t_generation)
    return get_unique_list(make),get_unique_list(mode),get_unique_list(submodel),get_unique_list(generation)
make,model,submodel,generation = create_unique_multitask_list()

In [39]:
print("make:",len(make))
print("model:",len(model))
print("submodel:",len(submodel))
print("generation:",len(generation))

make: 45
model: 338
submodel: 11
generation: 11


In [30]:
def separate_classes():
    arr = []
    for idx, annotation in enumerate(cars_annotations):
        make,model,submodel,generation = annotation.split()
        arr.append((idx, annotation,make, model, submodel, generation))
    arr = pd.DataFrame(arr, columns=['target', 'full_annotation','make', 'model', 'submodel', 'generation'])
    return arr
v2_info = separate_classes()

In [33]:
make_codes = v2_info['make'].astype('category').cat.codes
make_codes

0      35
1      35
2      35
3      35
4      35
       ..
688    14
689    31
690    41
691     7
692    28
Length: 693, dtype: int8

The hard split contains 107 fine-grained classes 
11 653 tracks (51 691 images) for training and 11 125 tracks (39 149 images)
for testing

In [6]:
import scipy.io
def load_anno(path):
    mat = scipy.io.loadmat(path)
    return mat

def load_class_names(path="/Users/martinodonnell/Documents/uni/fourth_year/4006/LowResImageRecognision/data/StanfordCars/devkit/cars_meta.mat"):
    cn = load_anno(path)['class_names']
    cn = cn.tolist()[0]
    cn = [str(c[0].item()) for c in cn]
    return cn

test = load_class_names()

In [7]:
test

['AM General Hummer SUV 2000',
 'Acura RL Sedan 2012',
 'Acura TL Sedan 2012',
 'Acura TL Type-S 2008',
 'Acura TSX Sedan 2012',
 'Acura Integra Type R 2001',
 'Acura ZDX Hatchback 2012',
 'Aston Martin V8 Vantage Convertible 2012',
 'Aston Martin V8 Vantage Coupe 2012',
 'Aston Martin Virage Convertible 2012',
 'Aston Martin Virage Coupe 2012',
 'Audi RS 4 Convertible 2008',
 'Audi A5 Coupe 2012',
 'Audi TTS Coupe 2012',
 'Audi R8 Coupe 2012',
 'Audi V8 Sedan 1994',
 'Audi 100 Sedan 1994',
 'Audi 100 Wagon 1994',
 'Audi TT Hatchback 2011',
 'Audi S6 Sedan 2011',
 'Audi S5 Convertible 2012',
 'Audi S5 Coupe 2012',
 'Audi S4 Sedan 2012',
 'Audi S4 Sedan 2007',
 'Audi TT RS Coupe 2012',
 'BMW ActiveHybrid 5 Sedan 2012',
 'BMW 1 Series Convertible 2012',
 'BMW 1 Series Coupe 2012',
 'BMW 3 Series Sedan 2012',
 'BMW 3 Series Wagon 2012',
 'BMW 6 Series Convertible 2007',
 'BMW X5 SUV 2007',
 'BMW X6 SUV 2012',
 'BMW M3 Coupe 2012',
 'BMW M5 Sedan 2010',
 'BMW M6 Convertible 2010',
 'BMW X3

In [ ]:
def load_annotations_v2(path, v2_info):
    ann = load_anno(path)['annotations'][0]
    ret = {}
    make_codes = v2_info['make'].astype('category').cat.codes
    type_codes = v2_info['model_type'].astype('category').cat.codes

    for idx in range(len(ann)):
        x1, y1, x2, y2, target, imgfn = ann[idx]

        r = {
            'x1': x1.item(),
            'y1': y1.item(),
            'x2': x2.item(),
            'y2': y2.item(),
            'target': target.item() - 1,
            'make_target': make_codes[target.item() - 1].item(),
            'type_target': type_codes[target.item() - 1].item(),
            'filename': imgfn.item()
        }

        ret[idx] = r
    return ret